# Read using pandas

In [ ]:
import pandas as pd
data = pd.read_csv('data/klej-polemo2-out/train.csv')

# Load using a library datasets

In [3]:
from pprint import pprint
from datasets import load_dataset

In [4]:
dataset = load_dataset("data/klej-polemo2-out")
pprint(dataset['train'][1])


# {'sentence': 'Super lekarz i człowiek przez duże C . Bardzo duże doświadczenie '
#              'i trafne diagnozy . Wielka cierpliwość do ludzi starszych . Od '
#              'lat opiekuje się moją Mamą staruszką , i twierdzę , że mamy duże '
#              'szczęście , że mamy takiego lekarza . Naprawdę nie wiem cobyśmy '
#              'zrobili , gdyby nie Pan doktor . Dzięki temu , moja mama żyje . '
#              'Każda wizyta u specjalisty jest u niego konsultowana i uważam , '
#              'że jest lepszy od każdego z nich . Mamy do Niego prawie '
#              'nieograniczone zaufanie . Można wiele dobrego o Panu doktorze '
#              'jeszcze napisać . Niestety , ma bardzo dużo pacjentów , jest '
#              'przepracowany ( z tego powodu nawet obawiam się o jego zdrowie ) '
#              'i dostęp do niego jest trudny , ale zawsze możliwy .',
#  'target': '__label__meta_plus_m'}


Using custom data configuration klej-polemo2-out-c71e8074a6696508
Found cached dataset csv (C:/Users/milos/.cache/huggingface/datasets/csv/klej-polemo2-out-c71e8074a6696508/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317)


  0%|          | 0/3 [00:00<?, ?it/s]

{'sentence': 'Bardzo olewcze podejscie do pacjenta . Przyprowadzajac dziecko z '
             'ostra wysypka na calym ciele trwajaca od 2 tygodni Pani doktor '
             'stwierdzila ze nie widzi wskazan wystawienia dziecku skierowania '
             'na testy skorne . Chocby na nasza prosbe i dodam iz w prywatnej '
             'klinice ( gdzie tak czy siak musielibysmy za to zaplacic ) '
             'Odmowila rowniez wystawienia zaswiadczenia o tym ze dziecko jest '
             '" zdrowe " i moze uczeszczac do przedszkola twierdzac ze takich '
             'zaswiadczen sie nie wystawia . L4 oczywiscie rozniez nie '
             'wchodzilo w gre , . Jednym slowem z gabinetu wyszlismy bez '
             'niczego mimo iz kierowani bylismy przez dwoch innych pediatrow '
             'na natychmiastowe testy do alergologa .',
 'target': '__label__meta_minus_m'}


In [8]:
dataset2 = load_dataset("data/aspectemo")
print(dataset2["train"][5])
print(len(dataset2["test"][3]['tokens']))
print(len(dataset2["test"][3]['labels']))

No config specified, defaulting to: aspectemo/1.0
Found cached dataset aspectemo (C:/Users/milos/.cache/huggingface/datasets/aspectemo/1.0/1.0.0/968ba95943dbf225ce5847233e07ecab3eb9b26791d66cb7ef72a5179503c625)


  0%|          | 0/2 [00:00<?, ?it/s]

{'tokens': ['Wykłady', 'prowadzone', 'dośc', 'monotonnie', ',', 'sprawiają', 'że', 'człowiek', 'ma', 'ochote', 'sie', 'przespac', '.', 'Obecnośc', 'niby', 'sprawdzana', '-', 'czasem', 'puszczał', 'ponoc', 'liste', ',', 'ale', 'na', 'koniec', 'nie', 'bierze', 'tego', 'pod', 'uwagę', '.', 'Ocena', 'z', 'egzaminu', '/', 'zaliczenia', '-', 'pytania', 'proste', ',', 'podał', 'je', 'wcześniej', '.', 'Kurs', 'może', 'zaciekawic', 'kogos', ',', 'kto', 'interesuje', 'się', 'filozofią', ',', 'ale', 'moim', 'zdaniem', 'był', 'strasznie', 'nudny', '.'], 'labels': [2, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 3, 0, 0, 0, 0, 3, 0, 0, 0, 0, 0, 0, 0, 3, 0, 0, 0, 0, 3, 0, 0, 0, 0, 0, 5, 0, 0, 5, 0, 0, 0, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0]}
139
139


### Evaluation

In [5]:
import random
from datasets import load_metric

In [6]:
dataset = dataset.class_encode_column("target")
references = dataset["test"]["target"]

# generate random predictions
predictions = [random.randrange(max(references) + 1) for _ in range(len(references))]

Loading cached processed dataset at C:/Users/milos/.cache/huggingface/datasets/csv/klej-polemo2-out-c71e8074a6696508/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317\cache-b0aba63abc377108.arrow
Loading cached processed dataset at C:/Users/milos/.cache/huggingface/datasets/csv/klej-polemo2-out-c71e8074a6696508/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317\cache-65bfa5f91a4fda06.arrow
Loading cached processed dataset at C:/Users/milos/.cache/huggingface/datasets/csv/klej-polemo2-out-c71e8074a6696508/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317\cache-28a8b04ab615e30a.arrow
Loading cached processed dataset at C:/Users/milos/.cache/huggingface/datasets/csv/klej-polemo2-out-c71e8074a6696508/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317\cache-767640a3a0bbd524.arrow
Loading cached processed dataset at C:/Users/milos/.cache/huggingface/datasets/csv/klej-polemo2-out-c71e8074a6696508/0.0.0/6b34fb8fc

In [7]:
acc = load_metric("accuracy")
f1 = load_metric("f1")

acc_score = acc.compute(predictions=predictions, references=references)
f1_score = f1.compute(predictions=predictions, references=references, average="macro")

pprint(acc_score)
pprint(f1_score)

# {'accuracy': 0.2894736842105263}
# {'f1': 0.2484406098784191}

C:\Users\milos\AppData\Local\Temp\ipykernel_11524\2252151753.py:1: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  acc = load_metric("accuracy")


{'accuracy': 0.2793522267206478}
{'f1': 0.23618658559756794}
